# Goji Data Compilation

<a href="https://colab.research.google.com/github/jasmine-schoch/goji-data-analysis/blob/main/goji_data_compilation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import needed libraries and define constants

In [35]:
import math
import numpy as np
import os
import pandas as pd
import pyarrow
from slugify import slugify

# Sheets file containing all of the links to individual company's information:
DATA_SOURCES_GOOGLE_SHEETS = "https://docs.google.com/spreadsheets/d/1oEaZVl3YF6JCxS63wsk10HO2dCVFs_CyfZ-cmlMFU0c/edit?usp=sharing"
GOOGLE_SHEETS_TO_CSV_SUFFIX = "/export?format=csv"
CACHE_DIR = "local_cache/"

## Initialize Variables and Data

### Define output data frame and columns

In [36]:
# Final data frame
Final = pd.DataFrame()
Final.index = [
    "Total",
    "Carbon Emmisions",
    "Water Usage",
    "Ethical Sourcing",
    "Labor Rights",
    "Transparency & Policy",
    "DEI",
]

### Get data sources URLs into a data frame

In [37]:
csv_url = DATA_SOURCES_GOOGLE_SHEETS[:DATA_SOURCES_GOOGLE_SHEETS.rindex('/')] + GOOGLE_SHEETS_TO_CSV_SUFFIX
data_source_urls_df = pd.read_csv(csv_url, header=None)

## Get all company data
Load data from local cache if available. Download the data otherwise.

In [45]:
# Dictionary of company name -> data frame
company_to_data = {}

# Create cache dir if needed
if not os.path.exists(CACHE_DIR):
    os.makedirs(CACHE_DIR, exist_ok=True)

# Iterate through each company and load the data
for index, row in data_source_urls_df.iterrows():
    # Define cache path
    cache_filename = CACHE_DIR + slugify(row[0])+".feather"

    # Define data url
    data: pd.DataFrame

    try:
        # Try loading the data locally
        data = pd.read_feather(cache_filename)
    except FileNotFoundError:
        # Get the data online
        online_url = row[1][:row[1].rindex('/')] + GOOGLE_SHEETS_TO_CSV_SUFFIX
        data = pd.read_csv(online_url, header=None)

        # Cache data (convert columns to strings)
        data.columns = data.columns.map(str)
        data.to_feather(cache_filename)
    
    # Record data frame into dictionary
    company_to_data[row[0]] = data

## Compile each company's data into `Final`

In [46]:
for company, data in company_to_data.items():
    # Pull info for each column
    total = int(data.iloc[68, 9])
    carbon = int(data.iloc[84, 9])
    water = int(data.iloc[85, 9])
    sourcing = sum(map(int, data.iloc[[80, 81], 9]))
    labor = sum(map(int, data.iloc[[71, 72, 73, 76, 79, 92], 9]))
    trans = sum(map(int, data.iloc[[69, 75, 83, 87, 89, 91, 92], 9]))
    dei = int(data.iloc[90, 9])

    # Save into Final data frame
    Final[data.iloc[2, 2]] = [total, carbon, water, sourcing, labor, trans, dei]

## Calculate percentile rankings

In [206]:
# Finding percentile rankings:
for x in range(len(Final)):  # looping through each row
    rowcurr = list(Final.iloc[x, :])  # setting row as a list
    length = len(rowcurr)
    for y in range(length):  # looping through each data point
        count = 0
        j = rowcurr[y]
        for z in range(length):
            if j > rowcurr[z]:
                count += 1
        Final.iloc[x, y] = int((math.trunc(round(((count / length) * 100)))))
for x in range(len(Final.columns)):
    Final.iloc[0, x] = int((math.trunc(round((sum(Final.iloc[1:, x]) / 6)))))

## Save the results to a JSON file

In [212]:
Reversed = Final.loc[
    [
        "Carbon Emmisions",
        "Water Usage",
        "Ethical Sourcing",
        "Labor Rights",
        "Transparency & Policy",
        "DEI",
        "Total",
    ]
]

# Saving final table to json
# Don't forget to export file from sidebar!!
Reversed.to_json("percentile.json", orient="split", compression="infer", index="true")